<a href="https://colab.research.google.com/github/RozForCode/Automation/blob/main/SeoulBikeSharing_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
from imblearn.over_sampling import RandomOverSampler
import copy
import seaborn as sns
import tensorflow as tf
from sklearn.linear_model import LinearRegression

In [5]:
dataset_cols = ["bike_count","hour","temp","humidity","wind","visibilitiy","dew_pt_temp","radiation","rain","snow","functional"]

In [ ]:
df = pd.read_csv('SeoulBikeData.csv').drop('Data','Holiday','Seasons', axis=1)
df.columns = dataset_cols
df["functional"] = [df["functional"]=='Yes'].astype(int)
df.drop('Hours')

In [ ]:
for thing in df.columns[1:]:
  plt.scatter(df[thing], df['bike_count'])
  plt.title(thing)
  plt.ylabel("Bike count at noon")
  plt.xlabel(thing)
#from this we'll see the ones that don't provide any relationship and drop them from df

In [ ]:
df.drop('columns that provide no relationship', axis=1)

In [ ]:
#now split into training, validation and test dataset
train, test, val = np.split(df.sample(frac=1), int(0.6*len(df)), int(0.8*len(df)))

In [ ]:
def get_xy(dataframe, y_label, x_label=None):
  dataf = copy.deepcopy(df)
  if not x_label:
    x = dataf( c for c in dataf.columns if c != y_label).values
  else:
    if len(x_label) == 1:
      x = dataf[x_label[0]].values.reshape(-1,1)
    else:
      x = dataf[x_label].values
  y = dataf[y_label].values.reshape(-1,1)
  return np.hstack((x,y)) , x, y

In [ ]:
full , x_train, y_train = get_xy(train, 'bike_count',x_label="temp")
full , x_test, y_test = get_xy(test, 'bike_count',x_label="temp")
full , x_val, y_val = get_xy(val, 'bike_count',x_label="temp")

In [ ]:
temp_reg1 = LinearRegression()
temp_reg1.fit(x_train, y_train)
print(temp_reg1.coef_ , temp_reg1.intercept_)

In [ ]:
temp_reg1.score(x_test, y_test)
#we can use plt to plot this singular Linear Regression

#multiple linear regression

In [ ]:
all , x_train_all, y_train_all = get_xy(train, 'bike_count',df.columns[1:])
all , x_test_all, y_test_all = get_xy(test, 'bike_count',df.columns[1:])
all , x_val_all, y_val_all = get_xy(val, 'bike_count',df.columns[1:])

In [ ]:
all_reg = LinearRegression()
all_reg.fit(x_train_all, y_train_all)

# reg with neural network

In [ ]:
temp_normalizer = tf.keras.layers.Normalization(input_shape=(1,), axis=None)#shape =1 cause singular with temp only
temp_normalizer.adapt(x_train.reshape(-1))

In [ ]:
temp_nn_modol = tf.keras.Sequential([
    temp_normalizer,
    tf.keras.layers.Dense(1)
])

In [ ]:
temp_nn_model.compile(optimizer = tf.keras.optimizer.Adam(learning_rate = 0.1) , loss= 'mean_sqaured_error' )

In [ ]:
history = temp_nn_model.fit(x_train.reshape(-1), y_train, verbose = 0, epochs = 1000, validation_data=(x_val, y_val))
# use plt to see the model